In [0]:
#!kill -9 -1

In [1]:
from itertools import chain
import random
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.metrics
import scipy as sp
import sklearn.linear_model
import tensorflow as tf
sess = tf.Session()
from keras import backend as K
from keras import *
from keras.layers import *
from keras.callbacks import LambdaCallback, Callback, ReduceLROnPlateau, TensorBoard
import sklearn.linear_model
from hyperas.distributions import uniform, choice
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
K.set_session(sess)

Using TensorFlow backend.


In [2]:
"""
Created on Tue Nov  6 10:06:52 2018

@author: yandexdataschool

Original Code found in:
https://github.com/yandexdataschool/roc_comparison

updated: Raul Sanchez-Vazquez
"""

import scipy.stats
from scipy import stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


In [3]:
def data():

    import pandas as pd

    column_names = ['A1',
                'A2',
                'A3',
                'A4',
                'A5',
                'A6',
                'A7',
                'A8',
                'A9',
                'A10',
                'A11',
                'A12',
                'A13',
                'A14',
                'A15',
               'A16',
               'A17',
               'A18',
               'A19',
               'A20',
               'target']

    german = pd.read_csv(
        "~/Google Drive/Discrétisation ICLR19/opendata/german.data",
        sep="\s",
        names = column_names,
        header= None,
        index_col = False,
        na_values=[
            '-NaN', '-nan',
            'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null',
            '.'
        ])
    
    german.target = german.target-1
    german.dropna(inplace=True)
    german.reset_index(inplace=True, drop=True)
    german_label_encoders = []

    german_encoded = german.copy()

    for j in ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']:
        temp = sk.preprocessing.LabelEncoder()
        temp.fit(german[j].astype(str))
        german_label_encoders.append(temp)
        german_encoded[j] = temp.transform(german[j].astype(str))
        
    german_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
    german_one_hot_encoder.fit(german_encoded[[
    'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'
    ]])
    german_one_hot_encoded = german_encoded.copy()
    german_one_hot_encoded.drop(
    ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'],
    axis=1,
    inplace=True)
    german_one_hot_encoded = pd.concat(
    [
    german_one_hot_encoded,
    pd.DataFrame(
    german_one_hot_encoder.transform(german_encoded[[
      'A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20'
    ]]),
    index=german_one_hot_encoded.index)
    ],
    axis=1)  
    german_features_train, german_features_test, german_perf_train, german_perf_test = sk.model_selection.train_test_split(
    german_one_hot_encoded.drop('target', axis=1),
    german_one_hot_encoded.target,
    test_size=0.33,
    random_state=1)
    german_nn_features_train = german_encoded.iloc[
    german_features_train.index, :].drop(
    'target', axis=1)
    german_nn_features_test = german_encoded.iloc[german_features_test.index, :].drop(
    'target', axis=1)
    german_nn_perf_train = german_encoded.iloc[
    german_features_train.index, :].target
    german_nn_perf_test = german_encoded.iloc[german_features_test.index, :].target

    
    global n
    global n_test
    n = german_nn_features_train.shape[0]
    n_test = german_nn_features_test.shape[0]



    x_quant = german_nn_features_train[[
    'A2','A5','A8','A11','A13','A16','A18'
    ]].values
    x_qual = german_nn_features_train[['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']].values
    y = german_nn_perf_train.values


    d1 = x_quant.shape[1]
    d2 = x_qual.shape[1]





    x_quant_test = german_nn_features_test[[
    'A2','A5','A8','A11','A13','A16','A18'
    ]].values
    x_qual_test = german_nn_features_test[['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']].values
    y_test = german_nn_perf_test.values


    for j in range(d2):
        liste_absents = [
        item for item in np.unique(x_qual_test[:, j])
        if item not in np.unique(x_qual[:, j])
        ]
        liste_lignes = [
        np.where(x_qual_test[:, j] == item) for item in liste_absents
        ]
        x_qual_test = np.delete(
        x_qual_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        x_quant_test = np.delete(
        x_quant_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        y_test = np.delete(
        y_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        
    x_qual_encoded = x_qual.copy()
    x_qual_label_encoders = []
    
    for j in range(d2):
        temp = sk.preprocessing.LabelEncoder()
        temp.fit(x_qual[:,j].astype(str))
        x_qual_label_encoders.append(temp)
        x_qual_encoded[:,j] = temp.transform(x_qual[:,j].astype(str))
        
    x_qual_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
    
    x_qual_one_hot_encoder.fit(x_qual_encoded)
    
    x_qual_dummy = x_qual_one_hot_encoder.transform(x_qual_encoded)
    
    x_qual = x_qual_encoded
    
    x_qual_test_encoded = x_qual_test.copy()
    
    for j in range(d2):
        x_qual_test_encoded[:,j] = x_qual_label_encoders[j].transform(x_qual_test[:,j].astype(str))
        
    x_qual_test = x_qual_test_encoded

    x_qual_dummy_test = x_qual_one_hot_encoder.transform(x_qual_test)


    return x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test


In [4]:
x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test = data()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [5]:
np.savetxt("x_quant.csv", x_quant, delimiter=",")
np.savetxt("x_qual.csv", x_qual, delimiter=",")
np.savetxt("y.csv", y, delimiter=",")
np.savetxt("x_quant_test.csv", x_quant_test, delimiter=",")
np.savetxt("x_qual_test.csv", x_qual_test, delimiter=",")
np.savetxt("y_test.csv", y_test, delimiter=",")

In [4]:
n = 670
n_test = 330
d1 = 7
d2 = 13

liste_layers_quant = [None] * d1
liste_layers_qual = [None] * d2
liste_qual_arrays = [None] * d2
liste_qual_arrays_test = [None] * d2

def create_model(x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test):    
    def initialize_neural_net(m_quant,m_qual):
        liste_inputs_quant = [None] * d1
        liste_inputs_qual = [None] * d2

        liste_layers_quant = [None] * d1
        liste_layers_qual = [None] * d2

        liste_layers_quant_inputs = [None] * d1
        liste_layers_qual_inputs = [None] * d2

        for i in range(d1):
            liste_inputs_quant[i] = Input((1, ))
            liste_layers_quant[i] = Dense(m_quant[i], activation='softmax')
            liste_layers_quant_inputs[i] = liste_layers_quant[i](
                liste_inputs_quant[i])

        for i in range(d2):
            liste_inputs_qual[i] = Input((len(np.unique(x_qual[:, i])), ))
            if (len(np.unique(x_qual[:, i])) > m_qual[i]):
                liste_layers_qual[i] = Dense(
                m_qual[i], activation='softmax', use_bias=False)
            else:
                liste_layers_qual[i] = Dense(
                len(np.unique(x_qual[:, i])), activation='softmax', use_bias=False)

            liste_layers_qual_inputs[i] = liste_layers_qual[i](
                liste_inputs_qual[i])

        return ([
            liste_inputs_quant, liste_layers_quant, liste_layers_quant_inputs,
            liste_inputs_qual, liste_layers_qual, liste_layers_qual_inputs
        ])
    
    
    def from_layers_to_proba_training(d1,d2,liste_layers_quant,liste_layers_qual):

        results = [None] * (d1 + d2)

        for j in range(d1):
            results[j] = K.function([liste_layers_quant[j].input],
                                    [liste_layers_quant[j].output])(
                                        [x_quant[:, j, np.newaxis]])

        for j in range(d2):
            results[j + d1] = K.function([liste_layers_qual[j].input],
                                         [liste_layers_qual[j].output])(
                                             [liste_qual_arrays[j]])

        return (results)
    
    
    
    
    def from_weights_to_proba_test(d1,d2,m_quant,m_qual,history,x_quant_test,x_qual_test,n_test):

        results = [None] * (d1 + d2)    

        for j in range(d1):
            results[j] = np.zeros((n_test,m_quant[j]))
            for i in range(m_quant[j]):
                results[j][:,i] = history.best_weights[j][1][i] + history.best_weights[j][0][0][i]*x_quant_test[:,j]


        for j in range(d2):
            results[j+d1] = np.zeros((n_test,history.best_weights[j+d1][0].shape[1]))
            for i in range(history.best_weights[j+d1][0].shape[1]):
                for k in range(n_test):
                    results[j+d1][k,i] = history.best_weights[j+d1][0][x_qual_test[k,j],i]

        return(results)
    
    
    def evaluate_disc(type,d1,d2,misc):
        if type=="train":
            proba = from_layers_to_proba_training(d1,d2,misc[0],misc[1])
        else:
            proba = from_weights_to_proba_test(d1,d2,misc[0],misc[1],misc[2],misc[3],misc[4],misc[5])


        results = [None] * (d1 + d2)

        if type=="train":
            X_transformed = np.ones((n, 1))
        else:
            X_transformed = np.ones((n_test, 1))

        for j in range(d1 + d2):
            if type=="train":
                results[j] = np.argmax(proba[j][0], axis=1)
            else:
                results[j] = np.argmax(proba[j], axis=1)
            X_transformed = np.concatenate(
                (X_transformed, sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore").fit_transform(
                    X=results[j].reshape(-1, 1))),
                axis=1)

        proposed_logistic_regression = sk.linear_model.LogisticRegression(
            fit_intercept=False, solver = "saga", C=1e20, tol=1e-8)


        if type=="train":
            proposed_logistic_regression.fit(X=X_transformed, y=y.reshape((n, )))
            performance = 2 * sk.metrics.log_loss(
              y,
              proposed_logistic_regression.predict_proba(X=X_transformed)[:, 1],
              normalize=False
          ) + proposed_logistic_regression.coef_.shape[1] * np.log(n)
            predicted = proposed_logistic_regression.predict_proba(X_transformed)[:,1]

        else:
            proposed_logistic_regression.fit(X=X_transformed, y=y_test.reshape((n_test, )))
            performance = 2*sk.metrics.roc_auc_score(y_test,proposed_logistic_regression.predict_proba(X_transformed)[:,1])-1
            predicted = proposed_logistic_regression.predict_proba(X_transformed)[:,1]

        return (performance, predicted)


    
    class LossHistory(Callback):
        def on_train_begin(self, logs={}):
            self.losses = []
            self.best_criterion = float("inf")
            self.best_outputs = []

        def on_epoch_end(self, batch, logs={}):
            self.losses.append(evaluate_disc("train",d1,d2,[liste_layers_quant,liste_layers_qual])[0])
            if self.losses[-1] < self.best_criterion:
                self.best_weights = []
                self.best_outputs = []
                self.best_criterion = self.losses[-1]
                for j in range(d1):
                    self.best_weights.append(liste_layers_quant[j].get_weights())
                    self.best_outputs.append(
                        K.function([liste_layers_quant[j].input],
                                   [liste_layers_quant[j].output])(
                                       [x_quant[:, j, np.newaxis]]))
                for j in range(d2):
                    self.best_weights.append(liste_layers_qual[j].get_weights())
                    self.best_outputs.append(
                        K.function([liste_layers_qual[j].input],
                                   [liste_layers_qual[j].output])(
                                       [liste_qual_arrays[j]]))
    
    
    
    essai = {{uniform(4,9)}}

    m_quant = [int(essai)] * d1
    m_qual = [int(essai)] * d2

    global liste_qual_arrays
    liste_qual_arrays = [None] * d2
    cursor = 0
    for j in range(d2):
        liste_qual_arrays[j] = x_qual_dummy[:, cursor:(
            cursor + len(np.unique(x_qual[:, j])))]
        cursor += len(np.unique(x_qual[:, j]))

    global liste_qual_arrays_test
    liste_qual_arrays_test = [None] * d2
    cursor = 0
    for j in range(d2):
        liste_qual_arrays_test[j] = x_qual_dummy_test[:, cursor:(
            cursor + len(np.unique(x_qual_test[:, j])))]
        cursor += len(np.unique(x_qual_test[:, j]))
  
  
    liste_inputs_quant = [None] * d1
    liste_inputs_qual = [None] * d2

    global liste_layers_quant
    global liste_layers_qual

    liste_layers_quant = [None] * d1
    liste_layers_qual = [None] * d2

    liste_layers_quant_inputs = [None] * d1
    liste_layers_qual_inputs = [None] * d2

    for i in range(d1):
        liste_inputs_quant[i] = Input((1, ))
        liste_layers_quant[i] = Dense(m_quant[i], activation='softmax')
        liste_layers_quant_inputs[i] = liste_layers_quant[i](
          liste_inputs_quant[i])

    for i in range(d2):
        liste_inputs_qual[i] = Input((len(np.unique(x_qual[:, i])), ))
        if (len(np.unique(x_qual[:, i])) > m_qual[i]):
            liste_layers_qual[i] = Dense(
            m_qual[i], activation='softmax', use_bias=False)
        else:
            liste_layers_qual[i] = Dense(
            len(np.unique(x_qual[:, i])), activation='softmax', use_bias=False)

        liste_layers_qual_inputs[i] = liste_layers_qual[i](
        liste_inputs_qual[i])


    full_hidden = concatenate(
        list(
          chain.from_iterable(
              [liste_layers_quant_inputs, liste_layers_qual_inputs])))
    output = Dense(1, activation='sigmoid')(full_hidden)
    model = Model(
      inputs=list(chain.from_iterable([liste_inputs_quant, liste_inputs_qual])),
      outputs=[output])  

    
    adam = optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = optimizers.SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    else:
        optim = sgd
        
 
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])

    history = LossHistory()

    callbacks = [
      ReduceLROnPlateau(
          monitor='loss',
          factor=0.5,
          patience=10,
          verbose=0,
          mode='auto',
          min_delta=0.0001,
          cooldown=0,
          min_lr=0), history
    ]
    model.fit(
      list(chain.from_iterable([list(x_quant.T), liste_qual_arrays])),
      y,
      epochs=500,
      batch_size={{choice([32,64,128])}},
      verbose=1,
      callbacks=callbacks)

    global n_test
    n_test = x_quant_test.shape[0]
    performance, predicted = evaluate_disc("test",d1,d2,misc=[m_quant,m_qual,history,x_quant_test,x_qual_test,n_test])
  
    K.clear_session()
    return {'loss': -performance, 'status': STATUS_OK, 'model': model, 'predicted': predicted}


In [5]:
trials = Trials()

best_run, best_model, space = optim.minimize(model=create_model,
                          data=data,
                          algo=tpe.suggest,
                          max_evals=10,
                          trials=trials,
                          notebook_name='hyperas - german - dump - ICML19',
                          eval_space=True,
                          return_space=True
)

>>> Imports:
#coding=utf-8

try:
    from itertools import chain
except:
    pass

try:
    import random
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import sklearn as sk
except:
    pass

try:
    import sklearn.metrics
except:
    pass

try:
    import scipy as sp
except:
    pass

try:
    import sklearn.linear_model
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras import *
except:
    pass

try:
    from keras.layers import *
except:
    pass

try:
    from keras.callbacks import LambdaCallback, Callback, ReduceLROnPlateau, TensorBoard
except:
    pass

try:
    import sklearn.linear_model
except:
    pass

try:
    from hyperas.distributions import uniform, choice
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
 

/Users/adrien/Google Drive/Colab Notebooks/temp_model.py:139: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  '.'


Epoch 1/500
670/670 [==============================] - 1s 1ms/step - loss: 0.6326 - acc: 0.6896


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Epoch 2/500
670/670 [==============================] - 0s 70us/step - loss: 0.6203 - acc: 0.6896
Epoch 3/500
670/670 [==============================] - 0s 81us/step - loss: 0.6206 - acc: 0.6896


...

Epoch 1/500
670/670 [==============================] - 2s 3ms/step - loss: 0.6242 - acc: 0.6896
Epoch 2/500
670/670 [==============================] - 0s 77us/step - loss: 0.6123 - acc: 0.6896
Epoch 3/500
670/670 [==============================] - 0s 54us/step - loss: 0.6104 - acc: 0.6896
Epoch 4/500
670/670 [==============================] - 0s 51us/step - loss: 0.5993 - acc: 0.6896
Epoch 5/500
670/670 [==============================] - 0s 61us/step - loss: 0.5948 - acc: 0.6896
Epoch 6/500
670/670 [==============================] - 0s 48us/step - loss: 0.5868 - acc: 0.6896
Epoch 7/500
670/670 [==============================] - 0s 53us/step - loss: 0.5841 - acc: 0.6881
Epoch 8/500
670/670 [==============================] - 0s 60us/step - loss: 0.5761 - acc: 0.6881
Epoch 9/500
670/670 [==============================] - 0s 55us/step - loss: 0.5659 - acc: 0.6896
Epoch 10/500
670/670 [==============================] - 0s 55us/step - loss: 0.5587 - acc: 0.6896
Epoch 11/500
670/670 [========

670/670 [==============================] - 0s 66us/step - loss: 0.4436 - acc: 0.7881
Epoch 168/500
670/670 [==============================] - 0s 70us/step - loss: 0.4424 - acc: 0.7881
Epoch 169/500
670/670 [==============================] - 0s 57us/step - loss: 0.4431 - acc: 0.7851
Epoch 170/500
670/670 [==============================] - 0s 75us/step - loss: 0.4433 - acc: 0.7821
Epoch 171/500
670/670 [==============================] - 0s 68us/step - loss: 0.4432 - acc: 0.7836
Epoch 172/500
670/670 [==============================] - 0s 69us/step - loss: 0.4428 - acc: 0.7866
Epoch 173/500
670/670 [==============================] - 0s 54us/step - loss: 0.4426 - acc: 0.7851
Epoch 174/500
670/670 [==============================] - 0s 53us/step - loss: 0.4424 - acc: 0.7896
Epoch 175/500
670/670 [==============================] - 0s 78us/step - loss: 0.4429 - acc: 0.7910
Epoch 176/500
670/670 [==============================] - 0s 50us/step - loss: 0.4427 - acc: 0.7896
Epoch 177/500
670/670 [=

670/670 [==============================] - 0s 57us/step - loss: 0.4422 - acc: 0.7881
Epoch 332/500
670/670 [==============================] - 0s 55us/step - loss: 0.4422 - acc: 0.7881
Epoch 333/500
670/670 [==============================] - 0s 54us/step - loss: 0.4422 - acc: 0.7881
Epoch 334/500
670/670 [==============================] - 0s 52us/step - loss: 0.4422 - acc: 0.7881
Epoch 335/500
670/670 [==============================] - 0s 89us/step - loss: 0.4422 - acc: 0.7881
Epoch 336/500
670/670 [==============================] - 0s 69us/step - loss: 0.4422 - acc: 0.7881
Epoch 337/500
670/670 [==============================] - 0s 69us/step - loss: 0.4422 - acc: 0.7881
Epoch 338/500
670/670 [==============================] - 0s 67us/step - loss: 0.4422 - acc: 0.7881
Epoch 339/500
670/670 [==============================] - 0s 56us/step - loss: 0.4422 - acc: 0.7881
Epoch 340/500
670/670 [==============================] - 0s 55us/step - loss: 0.4422 - acc: 0.7881
Epoch 341/500
670/670 [=

670/670 [==============================] - 0s 55us/step - loss: 0.4422 - acc: 0.7881
Epoch 496/500
670/670 [==============================] - 0s 53us/step - loss: 0.4422 - acc: 0.7881
Epoch 497/500
670/670 [==============================] - 0s 52us/step - loss: 0.4422 - acc: 0.7881
Epoch 498/500
670/670 [==============================] - 0s 57us/step - loss: 0.4422 - acc: 0.7881
Epoch 499/500
670/670 [==============================] - 0s 53us/step - loss: 0.4422 - acc: 0.7881
Epoch 500/500
670/670 [==============================] - 0s 51us/step - loss: 0.4422 - acc: 0.7881


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [6]:
print(best_run)

{'batch_size': 128, 'choiceval': 'adam', 'essai': 4.604510956047549, 'lr': 0.01, 'lr_1': 0.001, 'lr_2': 0.01}


In [7]:
trials.best_trial

{'state': 2,
 'tid': 9,
 'spec': None,
 'result': {'loss': -0.6922268907563025,
  'status': 'ok',
  'model': <keras.engine.training.Model at 0x1a6c6c64a8>,
  'predicted': array([0.92847789, 0.46241444, 0.04243385, 0.48004725, 0.48004725,
         0.52782865, 0.65752976, 0.16038152, 0.12293971, 0.04549465,
         0.85184944, 0.54459497, 0.05687207, 0.03319446, 0.22971375,
         0.18711339, 0.06433133, 0.02186368, 0.1350947 , 0.51146018,
         0.10160577, 0.58595088, 0.26101447, 0.05289267, 0.18299368,
         0.1061575 , 0.07457163, 0.07833728, 0.28134606, 0.92728209,
         0.09568621, 0.10115741, 0.21412641, 0.04561184, 0.41231068,
         0.8567388 , 0.05940928, 0.97251222, 0.3742719 , 0.05386353,
         0.24256783, 0.35596261, 0.01877736, 0.36896815, 0.18920973,
         0.1395347 , 0.31606119, 0.18053726, 0.03786007, 0.01291275,
         0.26891256, 0.13432116, 0.64493056, 0.38829746, 0.27959888,
         0.09820431, 0.72396138, 0.61272178, 0.09037446, 0.15372352,
   

In [8]:
trials.results

[{'loss': -0.6537267080745341,
  'status': 'ok',
  'model': <keras.engine.training.Model at 0x1a2b0403c8>,
  'predicted': array([0.49763604, 0.68159703, 0.05148485, 0.40555915, 0.5516377 ,
         0.71833719, 0.65081327, 0.28979508, 0.19296443, 0.07866662,
         0.64866154, 0.44386978, 0.03328362, 0.0763969 , 0.09000289,
         0.46992544, 0.05705325, 0.02357443, 0.15974946, 0.74831766,
         0.18086965, 0.55420019, 0.08983788, 0.05050538, 0.08214514,
         0.19505407, 0.40006067, 0.05095347, 0.30352089, 0.96433798,
         0.17744089, 0.0341788 , 0.17614507, 0.46641838, 0.33745347,
         0.8144697 , 0.02375996, 0.85397657, 0.33879181, 0.1476802 ,
         0.14946692, 0.36665393, 0.02931852, 0.2676072 , 0.16118219,
         0.13221276, 0.29314931, 0.29703862, 0.03790557, 0.06417992,
         0.35967833, 0.22939051, 0.72874933, 0.2789427 , 0.20098917,
         0.21195289, 0.76117192, 0.46817574, 0.09184919, 0.17170686,
         0.82684597, 0.06293866, 0.91882836, 0.05118

In [9]:
trials.best_trial['result']['predicted']

array([0.92847789, 0.46241444, 0.04243385, 0.48004725, 0.48004725,
       0.52782865, 0.65752976, 0.16038152, 0.12293971, 0.04549465,
       0.85184944, 0.54459497, 0.05687207, 0.03319446, 0.22971375,
       0.18711339, 0.06433133, 0.02186368, 0.1350947 , 0.51146018,
       0.10160577, 0.58595088, 0.26101447, 0.05289267, 0.18299368,
       0.1061575 , 0.07457163, 0.07833728, 0.28134606, 0.92728209,
       0.09568621, 0.10115741, 0.21412641, 0.04561184, 0.41231068,
       0.8567388 , 0.05940928, 0.97251222, 0.3742719 , 0.05386353,
       0.24256783, 0.35596261, 0.01877736, 0.36896815, 0.18920973,
       0.1395347 , 0.31606119, 0.18053726, 0.03786007, 0.01291275,
       0.26891256, 0.13432116, 0.64493056, 0.38829746, 0.27959888,
       0.09820431, 0.72396138, 0.61272178, 0.09037446, 0.15372352,
       0.39968267, 0.05374702, 0.71143795, 0.07581582, 0.70406262,
       0.46187141, 0.05465635, 0.05807356, 0.70142747, 0.57114722,
       0.24873065, 0.11184552, 0.02883648, 0.49360675, 0.31555

In [10]:
x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test=data()


alpha = .95
y_pred = trials.best_trial['result']['predicted']
y_true = y_test

auc, auc_cov = delong_roc_variance(
    y_true,
    y_pred)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Gini:', 2*auc-1)
print('AUC COV:', auc_cov)
print('95% Gini CI:', 2*ci-1)

Gini: 0.6922268907563027
AUC COV: 0.0005416495817667647
95% Gini CI: [0.60099697 0.78345681]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
